In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [58]:
# # naver map api key
# client_id =  '#개인 아이디
# client_pw = #개인 비밀번호

# api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# # 주소 목록 파일 (.csv)
# # data = pd.read_csv('파일명.csv')
# # 주소 목록 파일 (.xlsx)
# data = pd.read_excel('C:\geocodingtest\list_of_address.xlsx')

# # 네이버 지도 API 이용해서 위경도 찾기
# geo_coordi = []      # geographic coordinates
# for add in data['도로명주소']:
#     add_urlenc = parse.quote(add)  # 주소를 URL에서 사용할 수 있도록 URL Encoding
#     url = api_url + add_urlenc
#     request = Request(url)
#     request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
#     request.add_header('X-NCP-APIGW-API-KEY', client_pw)
#     try:
#         response = urlopen(request)
#     except HTTPError as e:
#         print('HTTP Error!')
#         latitude = None
#         longitude = None
#     else:
#         rescode = response.getcode() # 정상이면 200리턴
#         if rescode == 200:
#             response_body = response.read().decode('utf-8')
#             response_body = json.loads(response_body)   # json
#             if response_body['addresses'] == [] :
#                 print("'result' not exist!")
#                 latitude = None
#                 longitude = None
#             else:
#                 latitude = response_body['addresses'][0]['y']
#                 longitude = response_body['addresses'][0]['x']
#                 print("Success!")
#         else:
#             print('Response error code : %d' % rescode)
#             latitude = None
#             longitude = None

#     geo_coordi.append([latitude, longitude])


# np_geo_coordi = np.array(geo_coordi)
# pd_geo_coordi = pd.DataFrame({"구주소" :data['구주소'].values,\
#                                 "도로명": data['도로명주소'].values,\
#                               "위도": np_geo_coordi[:, 0],\
#                               "경도": np_geo_coordi[:, 1]})
# print(pd_geo_coordi)
# writer = pd.ExcelWriter('output_v2.xlsx')
# pd_geo_coordi.to_excel(writer, sheet_name='Sheet1')
# writer.save()
# # pd_geo_coordi.to_csv('C:\geocodingtest\result.csv')

In [6]:
# naver map api key
client_id =  ''#개인 아이디
client_pw =''#개인 비밀번호

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 크롤링에서 받아온 데이터를 정제하자. 예시데이터임
# 크롤링에서 받아온 구주소
address_gu = ['서울특별시 관악구 신림동 675-248번지', '서울특별시 관악구 신림동 1638-7번지',
              '서울특별시 관악구 신림동 94-107번지', '서울특별시 관악구 신림동 1735-2번지',
              '서울특별시 관악구 신림동 1578-36번지', '서울특별시 관악구 신림동 1669-6번지',
              '서울특별시 관악구 신림동 95-73번지', '서울특별시 관악구 신림동 92-251번지',
              '서울특별시 관악구 신림동 1536-2번지', '서울특별시 관악구 신림동 1423-21번지']
#크롤링에서 받아온 도로명 주소
address_do = ['서울특별시 관악구 난향9길  15-3', '서울특별시 관악구 신림로59길  15-10',
              '서울특별시 관악구 신림로36길  16', '서울특별시 관악구 난곡로  63',
              '서울특별시 관악구 남부순환로172길  16-7', '서울특별시 관악구 남부순환로  1391',
              '서울특별시 관악구 쑥고개로2가길  22', '서울특별시 관악구 신림로  248',
              '서울특별시 관악구 신림로17길  32', '서울특별시 관악구 봉천로12길  65']
no = np.arange(len(address_do))
data = np.array([no, address_gu, address_do])
data = data.T
print(data)
data_df = pd.DataFrame(data, columns=['No', '구주소', '도로명주소'])
data_df.head(10)

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []      # geographic coordinates
for add in data_df['도로명주소']:
    add_urlenc = parse.quote(add)  # 주소를 URL에서 사용할 수 있도록 URL Encoding
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode() # 정상이면 200리턴
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])


np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"구주소" :data_df['구주소'].values, 
                              "도로명": data_df['도로명주소'].values,
                              "위도": np_geo_coordi[:, 0], "경도": np_geo_coordi[:, 1]})
# print(pd_geo_coordi)
writer = pd.ExcelWriter('output_v2.xlsx')
pd_geo_coordi.to_excel(writer, sheet_name='Sheet1')
writer.save()

load_data = pd.read_excel('output_v2.xlsx')
address_point = pd.DataFrame(load_data)
latitude_point = address_point['위도']
longitude_point = address_point['경도']
pp = []
for point in zip(latitude_point, longitude_point) :
    pp.append(list(point))
print(pp)

[['0' '서울특별시 관악구 신림동 675-248번지' '서울특별시 관악구 난향9길  15-3']
 ['1' '서울특별시 관악구 신림동 1638-7번지' '서울특별시 관악구 신림로59길  15-10']
 ['2' '서울특별시 관악구 신림동 94-107번지' '서울특별시 관악구 신림로36길  16']
 ['3' '서울특별시 관악구 신림동 1735-2번지' '서울특별시 관악구 난곡로  63']
 ['4' '서울특별시 관악구 신림동 1578-36번지' '서울특별시 관악구 남부순환로172길  16-7']
 ['5' '서울특별시 관악구 신림동 1669-6번지' '서울특별시 관악구 남부순환로  1391']
 ['6' '서울특별시 관악구 신림동 95-73번지' '서울특별시 관악구 쑥고개로2가길  22']
 ['7' '서울특별시 관악구 신림동 92-251번지' '서울특별시 관악구 신림로  248']
 ['8' '서울특별시 관악구 신림동 1536-2번지' '서울특별시 관악구 신림로17길  32']
 ['9' '서울특별시 관악구 신림동 1423-21번지' '서울특별시 관악구 봉천로12길  65']]
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
[[37.4621748, 126.915345], [37.4827666, 126.9291672], [37.4759322, 126.9383139], [37.4607846, 126.9187534], [37.4832346, 126.9246739], [37.4804486, 126.905324], [37.4771804, 126.9393477], [37.4780957, 126.9341124], [37.4693477, 126.9363465], [37.485869, 126.9303342]]


In [55]:
pp

[[37.4621748, 126.915345],
 [37.4827666, 126.9291672],
 [37.4759322, 126.9383139],
 [37.4607846, 126.9187534],
 [37.4832346, 126.9246739],
 [37.4804486, 126.905324],
 [37.4771804, 126.9393477],
 [37.4780957, 126.9341124],
 [37.4693477, 126.9363465],
 [37.485869, 126.9303342]]

심곡동 삼겹살
https://m.map.naver.com/search2/search.naver?query=%EC%8B%AC%EA%B3%A1%EB%8F%99%20%EC%82%BC%EA%B2%B9%EC%82%B4&sm=shistory&style=v5

건대맛집
https://m.map.naver.com/search2/search.naver?query=%EA%B1%B4%EB%8C%80%20%EB%A7%9B%EC%A7%91&sm=hty&style=v5

https://m.map.naver.com/search2/search.naver?query=건대 치과&sm=hty&style=v5
https://m.map.naver.com/search2/search.naver?query=뚝섬 유원지&sm=hty&style=v5

건대%20맛집&sm=hty&style=v5
https://m.map.naver.com/search2/search.naver?query=%EA%B5%B0%EC%9E%90%20%EB%A7%9B%EC%A7%91&sm=hty&style=v5
군자%20맛집&sm=hty&style=v5